In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import time
import nlopt
import json
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.integrate import simps
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm

In [2]:
# Reading the CSV
df_curve_fit_organ_NLopt = pd.read_csv('df_fit_batch4_new.csv', index_col=0).sort_index()
# Convert JSON strings back to lists
df_curve_fit_organ_NLopt['Concentrations'] = df_curve_fit_organ_NLopt['Concentrations'].apply(json.loads)
df_curve_fit_organ_NLopt['Time_points'] = df_curve_fit_organ_NLopt['Time_points'].apply(json.loads)
df_curve_fit_organ_NLopt['Parameters'] = df_curve_fit_organ_NLopt['Parameters'].apply(json.loads)
# Convert list back to Numpy arrays
df_curve_fit_organ_NLopt['Concentrations'] = df_curve_fit_organ_NLopt['Concentrations'].apply(np.array)
df_curve_fit_organ_NLopt['Time_points'] = df_curve_fit_organ_NLopt['Time_points'].apply(np.array)
df_curve_fit_organ_NLopt['Parameters'] = df_curve_fit_organ_NLopt['Parameters'].apply(np.array)

print(df_curve_fit_organ_NLopt.shape)
df_curve_fit_organ_NLopt.head(5)

(1440, 7)


,Study_id,Instance,Organs,Concentrations,Time_points,Parameters,R2_score
0,1,1,Blood,"[83.5715121429407, 2.457985651262957, 2.001502...","[0.0, 0.16666666666666666, 1.0, 2.0, 4.0, 24.0...","[81.5329817362303, 31.679318243723753, -0.7734...",0.999998
1,1,1,Brain,"[0.0, 8.425676874874718, 2.9197890160456903, 2...","[0.0, 1.0, 24.0, 168.0]","[7.011668382413655, 0.09799548744597017, -9.26...",1.000000
2,1,1,Kidneys,"[0.0, 7.995076681233775, 2.884821482919395, 2....","[0.0, 1.0, 24.0, 168.0]","[-9.831295737175694, 2.719643487958578, 7.0288...",1.000000
3,1,1,Liver,"[0.0, 3.4513788334293967, 9.20367688914503, 0....","[0.0, 1.0, 24.0, 168.0]","[-15.192545150910798, 0.2847971466760059, 15.0...",1.000000
4,1,1,Lungs,"[0.0, 7.259475430305388, 4.619666182921618, 56...","[0.0, 1.0, 24.0, 168.0]","[1.9964470115182613, -0.019760492420510832, 1....",0.987680


In [13]:
df_curve_fit_organ_loaded = df_curve_fit_organ_NLopt.copy()

In [4]:
# Double exponential decay with c
def double_exponential_decay(t, a1, b1, a2, b2, c):
    return a1 * np.exp(-b1 * t) + a2 * np.exp(-b2 * t) + c

In [5]:
# Exponential decay
def exponential_decay(t, a1, b1, c):
    return a1 * np.exp(-b1 * t) + c

In [6]:
# Double exponential decay without c
def double_exponential_decay_net(t, a1, b1, a2, b2):
    return a1 * np.exp(-b1 * t) + a2 * np.exp(-b2 * t)

In [7]:
# Function RMSE
def rmse(predictions, observations):
    """
    Calculate the Root Mean Squared Error (RMSE).

    Parameters:
    predictions (list or np.array): Predicted values.
    observations (list or np.array): Observed values.

    Returns:
    float: The Root Mean Squared Error.
    """
    # Ensure inputs are numpy arrays
    y_obs = np.array(observations)
    y_pred = np.array(predictions)

    return np.sqrt(np.mean((y_pred - y_obs) ** 2))

In [8]:
# Function AAFE
def aafe(predictions, observations):
    """
    Calculate the Absolute Average Fold Error (AAFE).

    Parameters:
    predictions (list or np.array): Predicted values.
    observations (list or np.array): Observed values.

    Returns:
    float: The Absolute Average Fold Error.
    """
    if any(predictions)<0:
        return 1e06
    else:
        observations = [x + 1e-10 if x == 0 else x for x in observations]
        predictions = [x + 1e-10 if x == 0 else x for x in predictions]
        # Ensure inputs are numpy arrays
        y_obs = np.array(observations)
        y_pred = np.array(predictions)

        #Total number of observations
        N = len(y_obs)

        #Calculate log ratios
        log_ratio = np.abs(np.log10(y_pred/ y_obs))


        #Calculate AAFE
        aafe = 10 ** (np.sum(log_ratio) / N)

        return aafe

In [9]:
# Objective function for double exponential decay
def objective_function(x, grad, time_data, concentration_data, metric):
    a1, b1, a2, b2, c = x
    model_function = double_exponential_decay(time_data, *x)
    loss = aafe(model_function, concentration_data) if metric == "AAFE" else rmse(model_function, concentration_data)
    return loss

In [11]:
# Create a list with index of all Organ data
index_organ_list = []

for index, row in df_curve_fit_organ_loaded.iterrows():
    if row['Organs'] != 'Blood' and row['R2_score'] <= 0.96:
        index_organ_list.append(index)

index_organ_list.sort()
print(len(index_organ_list))

379


In [14]:
# Update the parameters in batches

# Calculate the length of each batch
batch_length = len(index_organ_list) // 4

# Divide the list into four sublists
batch_1 = index_organ_list[:batch_length]
batch_2 = index_organ_list[batch_length:2*batch_length]
batch_3 = index_organ_list[2*batch_length:3*batch_length]
batch_4 = index_organ_list[3*batch_length+50:]

# Print the lengths of the sublists
print(f"Batch_1 has {len(batch_1)} sets of parameters for optimization")
print(f"Batch_2 has {len(batch_2)} sets of parameters for optimization")
print(f"Batch_3 has {len(batch_3)} sets of parameters for optimization")
print(f"Batch_4 has {len(batch_4)} sets of parameters for optimization")

Batch_1 has 94 sets of parameters for optimization
Batch_2 has 94 sets of parameters for optimization
Batch_3 has 94 sets of parameters for optimization
Batch_4 has 47 sets of parameters for optimization


In [15]:
# Set the batch number
batch_list = batch_4
# Set number of evaluations
maxeval = 600
# Set relative tolerance
tolerance = 1e-4
# Set R2 score tolerance
r2_score_tolerance = 0.96

In [16]:
params_values = [-5,-1,0,1,5]

warnings.filterwarnings("ignore")


start_time = time.time()
total_iterations = len(batch_list)
pbar = tqdm(total=total_iterations, desc="Fitting curves for Organs", leave=False)


for index in batch_list:
    metric = 'AAFE' #"RMSE" OR "AAFE"
    metric_optimal = 1e+012
    # Define the experimental data
    time_data = df_curve_fit_organ_loaded.at[index, 'Time_points'] 
    concentration_data = df_curve_fit_organ_loaded.at[index, 'Concentrations'] 

    # Find the right parameters
    for a1 in params_values:
        for b1 in params_values:
            for a2 in params_values:
                for b2 in params_values:
                    for c in params_values:
                        try:

                            initial_params = [a1, b1, a2, b2, c]
                            opt = nlopt.opt(nlopt.LN_SBPLX, 5) # define the algorithm and parameters for optimization
                            opt.set_min_objective(lambda params, grad: objective_function(params, grad, time_data, concentration_data, metric))
                            #opt.set_min_objective(lambda params, grad: objective_function(params, time_data, concentration_data))
                            opt.set_xtol_rel(tolerance)  # Relative tolerance on the optimization
                            opt.set_maxeval(maxeval)   # Maximum number of function evaluations
                            optimized_params = opt.optimize(initial_params)

                            concentration_pred = double_exponential_decay(time_data, *optimized_params)

                            if metric == "AAFE":
                                metric_new = aafe(concentration_pred, concentration_data)
                            elif metric == "RMSE":
                                metric_new = rmse(concentration_pred, concentration_data)

                            if metric_new < metric_optimal:
                                new_params = optimized_params
                                metric_optimal = metric_new

                        except Exception:
                            pass
                            

    # Replace with the updated parameters
    df_curve_fit_organ_loaded.at[index, 'Parameters'] = new_params
    

    # Replace with the updated R2_score
    concentration_pred = double_exponential_decay(time_data, *new_params)
    R2_score = r2_score(concentration_data, concentration_pred)
    df_curve_fit_organ_loaded.at[index, 'R2_score'] = R2_score

    pbar.update(1)
pbar.close()
end_time = time.time()
elapsed_time_seconds = end_time - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")


Elapsed time: 39.43 minutes


In [ ]:
with PdfPages('organ_plots_AAFE_batch4_new.pdf') as pdf:
    for index in batch_list:
        
        time_data = df_curve_fit_organ_loaded.at[index, 'Time_points']
        concentration_data = df_curve_fit_organ_loaded.at[index, 'Concentrations']
        
        time_curve = np.linspace(time_data.min(),time_data.max(), 1000)
       

        concentration_curve = double_exponential_decay(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
                
            
        
 
        # Plot the data points
        plt.scatter(time_data, concentration_data, label='Experimental data')
        plt.plot(time_curve, concentration_curve, color='red', label='Fitted Curve')
        # Add labels and legend
        plt.xlabel('Time points (hours)')
        plt.ylabel('Concentrations (%ID/g)')
        plt.title(f"Index:{index}, Study_id:{df_curve_fit_organ_loaded.at[index, 'Study_id']}, Instance:{df_curve_fit_organ_loaded.at[index, 'Instance']}, Organ:{df_curve_fit_organ_loaded.at[index, 'Organs']}, R2: {df_curve_fit_organ_loaded.at[index, 'R2_score']}")
        plt.legend()

        # Save the plot to the PDF file
        pdf.savefig()
        plt.close()       

In [40]:
df_curve_fit_organ_save = df_curve_fit_organ_loaded.copy()

# Convert NumPy arrays to lists and then to JSON strings
df_curve_fit_organ_save['Concentrations'] = df_curve_fit_organ_save['Concentrations'].apply(lambda x: json.dumps(x.tolist()))
df_curve_fit_organ_save['Time_points'] = df_curve_fit_organ_save['Time_points'].apply(lambda x: json.dumps(x.tolist()))
df_curve_fit_organ_save['Parameters'] = df_curve_fit_organ_save['Parameters'].apply(lambda x: json.dumps(x.tolist()))

# Save to CSV
df_curve_fit_organ_save.to_csv('df_fit_batch4_new.csv', index=True)

In [31]:
organ_list = []

for index, row in df_curve_fit_organ_loaded.iterrows():
    if row['Organs'] != 'Blood':
        organ_list.append(index)

organ_list.sort()
print(len(organ_list))

1278


In [32]:
blood_list = []

for index, row in df_curve_fit_organ_loaded.iterrows():
    if row['Organs'] == 'Blood':
        blood_list.append(index)

blood_list.sort()
print(len(blood_list))

162


In [33]:
with PdfPages('organ_plots_FINAL.pdf') as pdf:
    for index in organ_list:
        
        # Define the experimental data 
        time_data = df_curve_fit_organ_loaded.at[index, 'Time_points'] # x-axis -> Time_points (hours)
        concentration_data = df_curve_fit_organ_loaded.at[index, 'Concentrations'] # y-axis -> concentrations (%ID/g)
        
        # Define curve fit data
        time_curve = np.linspace(time_data.min(),time_data.max(), 1000)
        
        if len(df_curve_fit_organ_loaded.at[index, 'Parameters']) <= 4:
            concentration_curve = exponential_decay(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
        else:
            concentration_curve = double_exponential_decay(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
 
        # Plot the data points
        plt.scatter(time_data, concentration_data, label='Experimental data')
        plt.plot(time_curve, concentration_curve, color='red', label='Fitted Curve')
        # Add labels and legend
        plt.xlabel('Time points (hours)')
        plt.ylabel('Concentrations (%ID/g)')
        plt.title(f"Index:{index}, Study_id:{df_curve_fit_organ_loaded.at[index, 'Study_id']}, Instance:{df_curve_fit_organ_loaded.at[index, 'Instance']}, Organ:{df_curve_fit_organ_loaded.at[index, 'Organs']}, R2: {df_curve_fit_organ_loaded.at[index, 'R2_score']}")
        plt.legend()

        # Save the plot to the PDF file
        pdf.savefig()
        plt.close()       

In [34]:
with PdfPages('blood_plots_FINAL.pdf') as pdf:
    for index in blood_list:
        
        # Define the experimental data 
        time_data = df_curve_fit_organ_loaded.at[index, 'Time_points'] # x-axis -> Time_points (hours)
        concentration_data = df_curve_fit_organ_loaded.at[index, 'Concentrations'] # y-axis -> concentrations (%ID/g)
        
        # Define curve fit data
        time_curve = np.linspace(time_data.min(),time_data.max(), 1000)
        
        if len(df_curve_fit_organ_loaded.at[index, 'Parameters'])==3:
            concentration_curve = exponential_decay(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
            
        elif len(df_curve_fit_organ_loaded.at[index, 'Parameters'])==4:
            concentration_curve = double_exponential_decay_net(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
            
        else:
            concentration_curve = double_exponential_decay(time_curve, *df_curve_fit_organ_loaded.at[index, 'Parameters'])
 
        # Plot the data points
        plt.scatter(time_data, concentration_data, label='Experimental data')
        plt.plot(time_curve, concentration_curve, color='red', label='Fitted Curve')
        # Add labels and legend
        plt.xlabel('Time points (hours)')
        plt.ylabel('Concentrations (%ID/g)')
        plt.title(f"Index:{index}, Study_id:{df_curve_fit_organ_loaded.at[index, 'Study_id']}, Instance:{df_curve_fit_organ_loaded.at[index, 'Instance']}, Organ:{df_curve_fit_organ_loaded.at[index, 'Organs']}, R2: {df_curve_fit_organ_loaded.at[index, 'R2_score']}")
        plt.legend()

        # Save the plot to the PDF file
        pdf.savefig()
        plt.close()       